In [ ]:
!pip3 install requests
!pip3 install BeautifulSoup4
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup

!pip install chardet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandas.core.internals.array_manager import NullArrayProxy
year = '2021'
flg = False
List = []
l=["01","02","03","04","05","06","07","08","09","10"]#競馬場
for w in range(len(l)):
  for z in range(7):#開催
    for y in range(14):#日
      if y<9:
        url1="https://db.netkeiba.com/race/"+year+l[w]+'0'+str(z+1)+'0'+str(y+1)
      else:
        url1="https://db.netkeiba.com/race/"+year+l[w]+'0'+str(z+1)+str(y+1)

      for x in range(12):
        if x<9:
          url=url1+str("0")+str(x+1)+'/'
        else:
          url=url1+str(x+1)+'/'
        res = requests.get(url)
        print(url)
        time.sleep(1)
        soup = BeautifulSoup(res.content,'html.parser')
        ybreak = soup.find_all('body',attrs={'id': 'db_race_detail'})

        #取得作業

        alldata = []

        #レース情報
        raceinfoall = soup.find('dd')
        raceclass = soup.find_all('p',attrs={'class': 'smalltxt'})
        if raceclass == []:
          flg = True
          print('xループを抜けます')
          break
        else:
          raceclass = raceclass[0]
          clas = raceclass.text.split(' ')[2].split('\xa0')[0]
          raceinfo = raceinfoall.find_all('span')[0]
          sur = raceinfo.text[0]
          rou = raceinfo.text[1]
          dis = raceinfo.text[2:6]
          con = raceinfo.text.split("/")[2].split(":")[1][1]
          info = {}
          info['クラス'] = clas
          info['コース'] = sur
          info['周り'] = rou
          info['距離'] = dis
          info['馬場状態'] = con
          info['レースコード'] = str(url)[29:-1]
          #レース結果
          race = soup.find_all('table',attrs={'class': 'race_table_01 nk_tb_common'})
          race = race[0]
          #馬情報
          horses = race.find_all('tr')
          horses.pop(0)
          for horse in horses:
              #データ部分
              data = horse.find_all('td')
              #着順
              result = data[0]
              result = result.text
              #枠番
              num1 = data[1]
              num1 = num1.text
              #馬番
              num2 = data[2]
              num2 = num2.text
              #馬名
              name = data[3]
              name = name.text.replace('\n','')
              #性
              sex = data[4]
              sex1 = sex.text[0]
              #齢
              age = sex.text[1]
              #斤量
              wei = data[5]
              wei = wei.text
              #騎手
              jo = data[6]
              jo = jo.text.replace('\n','')
              #タイム
              tim = data[7]
              tim = tim.text
              #着差
              dif = data[8]
              dif = dif.text
              #通過
              pas = data[10]
              pas = pas.text
              #上がり
              last = data[11]
              last = last.text
              #単勝
              odds = data[12]
              odds = odds.text
              #人気
              fav = data[13]
              fav = fav.text
              #馬体重
              hwei = data[14]
              hweig = hwei.text[:3]
              #増減
              zou = hwei.text[4:-1]

              raceres = {}
              raceres['着順'] = result
              raceres['枠番'] = num1
              raceres['馬番'] = num2
              raceres['馬名'] = name
              raceres['性'] = sex1
              raceres['齢'] = age
              raceres['斤量'] = wei
              raceres['騎手'] = jo
              raceres['タイム'] = tim
              raceres['着差'] = dif
              raceres['通過'] = pas
              raceres['上がり'] = last
              raceres['単勝'] = odds
              raceres['人気'] = fav
              raceres['馬体重'] = hweig
              raceres['増減'] = zou
              raceres.update(info)
              alldata.append(raceres)
          List.extend(alldata)
          print(l[w]+'競馬場'+str(z+1)+'回'+str(y+1)+'日'+str(x+1)+'R')
      if flg == True:
        flg = False
        print('yループを抜けます')
        break
    



In [ ]:
import pandas as pd
df = pd.DataFrame(List)
df

,着順,枠番,馬番,馬名,性,齢,斤量,騎手,タイム,着差,...,単勝,人気,馬体重,増減,クラス,コース,周り,距離,馬場状態,レースコード
0,1,5,9,ディーバサンライズ,牝,3,52,秋山稔樹,1:08.7,,...,13.5,7,474,-22,3歳未勝利,芝,右,1200,良,202101010101
1,2,3,6,バンベルク,牡,3,56,横山和生,1:08.8,3/4,...,3.9,2,470,-2,3歳未勝利,芝,右,1200,良,202101010101
2,3,7,14,リツィタル,牝,3,54,大野拓弥,1:08.9,1/2,...,124.7,14,394,+2,3歳未勝利,芝,右,1200,良,202101010101
3,4,5,10,クレマチステソーロ,牝,3,54,鮫島克駿,1:09.0,1/2,...,7.6,5,468,+2,3歳未勝利,芝,右,1200,良,202101010101
4,5,3,5,ペイシャケイティー,牝,3,54,古川吉洋,1:09.0,ハナ,...,7.5,4,456,+8,3歳未勝利,芝,右,1200,良,202101010101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47816,7,2,2,ジオルティ,牡,3,51,角田大和,2:42.0,1.1/4,...,49.6,10,502,+8,3歳以上1勝クラス,芝,右,2600,良,202110040812
47817,8,7,9,ロックグラス,牡,4,57,幸英明,2:42.2,1.1/4,...,43.7,8,464,0,3歳以上1勝クラス,芝,右,2600,良,202110040812
47818,9,7,8,サマーカナロア,牡,3,53,富田暁,2:42.2,ハナ,...,46.5,9,502,-10,3歳以上1勝クラス,芝,右,2600,良,202110040812
47819,10,8,10,レッドレイル,セ,5,57,浜中俊,2:42.4,1.1/4,...,14.0,6,436,+2,3歳以上1勝クラス,芝,右,2600,良,202110040812


In [ ]:
df.to_csv('2021-1.csv', index=None, line_terminator='\n',encoding = 'utf_8_sig')

In [ ]:
df1 = pd.read_csv('2021-1.csv')
df1.head(3)

,着順,枠番,馬番,馬名,性,齢,斤量,騎手,タイム,着差,...,単勝,人気,馬体重,増減,クラス,コース,周り,距離,馬場状態,レースコード
0,1,5,9,ディーバサンライズ,牝,3,52.0,秋山稔樹,1:08.7,NaN,...,13.5,7.0,474,-22.0,3歳未勝利,芝,右,1200,良,202101010101
1,2,3,6,バンベルク,牡,3,56.0,横山和生,1:08.8,3/4,...,3.9,2.0,470,-2.0,3歳未勝利,芝,右,1200,良,202101010101
2,3,7,14,リツィタル,牝,3,54.0,大野拓弥,1:08.9,1/2,...,124.7,14.0,394,2.0,3歳未勝利,芝,右,1200,良,202101010101
